In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
%run /Workspace/Users/eshivee@gmail.com/Atlikon_SportsBar_Data_Pipeline/Set_Up/utilities

In [0]:
bronze_schema, silver_schema

In [0]:
dbutils.widgets.text('catalog', 'sportats', 'Catalog')
catalog = dbutils.widgets.get('catalog')
dbutils.widgets.text('data_source', 'orders', 'Data Source')
data_source = dbutils.widgets.get('data_source')

basepath = f's3://sportsbarsa/{data_source}'
landing_path = f'{basepath}/landing'
processed_path = f'{basepath}/processed'
print(f'basepath: {basepath}')
print(f'landing_path: {landing_path}')
print(f'processed_path: {processed_path}')

bronze_table = f"{catalog}.{bronze_schema}.{data_source}"
silver_table = f"{catalog}.{silver_schema}.{data_source}"
gold_table = f"{catalog}.{gold_schema}.sb_fact_{data_source}"

In [0]:
df = spark.read.format('csv')\
    .option('header', True)\
    .option('inferSchema', True)\
    .load(f'{processed_path}/*.csv')\
    .withColumn('read_timestamp', current_timestamp())\
    .select('*' , '_metadata.file_name', '_metadata.file_size')

#this load path should be landing, but because I had already moved the files to procooessed path that why I used processed path

In [0]:
print('total_rows', df.count())
df.display()

In [0]:
df.write.format('delta')\
        .mode('append')\
        .option('delta.enableChangeDataFeed', 'true')\
        .saveAsTable(bronze_table)

In [0]:
files = dbutils.fs.ls(landing_path)

for file in files:
    dbutils.fs.mv(
        file.path,
        f'{processed_path}/{file.name}',
        True
    )

**SILVER FACT TABLE ORDERS**

In [0]:
df_bronze = spark.read.table(bronze_table)
df_bronze.display()

In [0]:
print('rows_with_null_qty', df_bronze.count())

df_silver = df_bronze.filter(col('order_qty').isNotNull())

print('rows_after_removing_nulls:',df_silver.count())

In [0]:
#lets take care of the customer ids that are not numeric

df_silver = df_silver.withColumn('customer_id', when(col("customer_id").rlike('^[0-9]+$'), col("customer_id"))\
                .otherwise('999999').cast('string'))


In [0]:
#lets now remove the rows that have day of the week before the actual date

df_silver = df_silver.withColumn(
    'order_placement_date',
    regexp_replace(col("order_placement_date"), r"^[A-Za-z]+,\s*", "")
)

df_silver.display()

In [0]:
df_silver = df_silver.withColumn(
    'order_placement_date',
    coalesce(
        try_to_date('order_placement_date', 'yyyy/MM/dd'),
        try_to_date('order_placement_date', 'dd-MM-yyyy'),
        try_to_date('order_placement_date', 'dd/MM/yyyy'),
        try_to_date('order_placement_date', 'MMMM dd, yyyy')
    )
)

In [0]:
df_silver.count()

In [0]:
df_silver = df_silver.dropDuplicates(subset =['order_id', 'order_placement_date', 'customer_id', 'product_id', 'order_qty'])

df_silver.count()

In [0]:
df_silver = df_silver.withColumn('product_id', col('product_id').cast('string'))

df_silver.printSchema()

In [0]:
df_silver.select(min('order_placement_date'), max('order_placement_date')).display()

In [0]:
df_products = spark.read.table('sportats.silver.products')

In [0]:
df_silver.display()

In [0]:
df_joined = df_silver.join(df_products.select('product_id', "product_code"), on = 'product_id', how = 'inner')

In [0]:
if spark.catalog.tableExists(silver_table):
    delta_obj = DeltaTable.forName(spark, silver_table)
    delta_obj.alias('trg').merge(df_joined.alias('src'),
                "trg.order_placement_date = src.order_placement_date AND\
                trg.order_id = src.order_id AND\
                trg.product_code = src.product_code AND\
                trg.customer_id = src.customer_id"              
                ).whenMatchedUpdateAll()\
                .whenNotMatchedInsertAll()\
                .execute()

#technically, we are merging the bronze df with the silver table. with the bronze table being the source and the silver table being the target.
else:
    df_joined.write.mode('overwrite')\
        .format('delta')\
        .option('delta.enableChangeDataFeed', 'true')\
        .option('mergeSchema', 'true')\
        .saveAsTable(silver_table)

In [0]:
spark.read.table(silver_table).display()

**GOLD LAYER TABLE ORDERS**

In [0]:
df_silver = spark.read.table(silver_table)

In [0]:
df_gold = df_silver.select(col('order_id'), col("order_placement_date").alias('date'), col("customer_id").alias('customer_code'), col('product_code'), col('product_id'), col("order_qty").alias('sold_quantity'))
df_gold.limit(10).display()

In [0]:
if spark.catalog.tableExists(gold_table):
    delta_obj = DeltaTable.forName(spark, gold_table)
    delta_obj.alias('trg').merge(df_gold.alias('src'),
                "trg.date = src.date AND\
                trg.order_id = src.order_id AND\
                trg.product_code = src.product_code AND\
                trg.customer_id = src.customer_id"              
                ).whenMatchedUpdateAll()\
                .whenNotMatchedInsertAll()\
                .execute()

#technically, we are merging the bronze df with the silver table. with the bronze table being the source and the silver table being the target.
else:
    df_gold.write.mode('overwrite')\
        .format('delta')\
        .option('delta.enableChangeDataFeed', 'true')\
        .option('mergeSchema', 'true')\
        .saveAsTable(gold_table)

In [0]:
spark.read.table(gold_table).display()

In [0]:
df_child_gold =  spark.read.table(gold_table)
df_child_gold = df_child_gold.select('date', 'product_code', 'customer_code', 'sold_quantity')

In [0]:
df_child_gold.count()

In [0]:
df_child_gold = df_child_gold.withColumn('date', trunc(col('date'), 'MM'))\
                            .groupBy('date', 'product_code', 'customer_code').agg(sum(col("sold_quantity")).alias('sold_quantity'))

df_child_gold.display()

#at the parent table, each record was a sum of each product bought by each customer for a given month. That was why we truncated the date column here, so that each month begins on the first day. 2025-12-18 --> 2025-12-01

In [0]:
p_table = spark.read.table('sportats.gold.fact_orders')

In [0]:
#lets now merge with the parent company

print('total_rows_before_merge:', p_table.count())
dlt_obj = DeltaTable.forName(spark, 'sportats.gold.fact_orders')
dlt_obj.alias('trg').merge(df_child_gold.alias('src'), "trg.date = src.date AND\
                                                        trg.product_code = src.product_code AND\
                                                        trg.customer_code = src.customer_code")\
                                                        .whenMatchedUpdateAll()\
                                                        .whenNotMatchedInsertAll()\
                                                        .execute()

#trg is the parent company and src is the child company

print('total_rows_after_merge:', p_table.count())

#upon reload. no new data was added. hence 96115 --> 96115, initally it was 96055